In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import geopandas as gpd
from pyproj import CRS

# Load the NetCDF file
file_path = "/shared_space/ACV/extracted_raw/yearly_files/yearly2003.nc"  # Replace with the desired file
ds = xr.open_dataset(file_path)

# Extract U10 and V10 variables
U10 = ds["U10"]
V10 = ds["V10"]
lons = ds["XLONG"]
lats = ds["XLAT"]

# # Compute wind speed and wind direction
# wind_speed = np.sqrt(U10**2 + V10**2)  # Total wind speed
# wind_direction = np.arctan2(V10, U10) * (180 / np.pi)  # Convert to degrees

# # Normalize wind direction to 0-360 degrees
# wind_direction = (wind_direction + 360) % 360

# # Select the first time step
# wind_speed_data = wind_speed.isel(Time=0)

# Lambert Conformal CRS parameters (use parameters from your dataset)
lambert_crs = ccrs.LambertConformal(
    central_longitude=-100.0,  # Replace with your dataset's central longitude
    central_latitude=35.0,     # Replace with your dataset's central latitude
    standard_parallels=(30.0, 60.0)  # Replace with standard parallels from your dataset
)

# Load and reproject the shapefile
shapefile_path = "/data/HW/HW2/NCDOT_County_Boundaries.shp"  # Replace with your actual shapefile path
counties = gpd.read_file(shapefile_path)

# Convert shapefile CRS to match Lambert Conformal from the NetCDF
netcdf_crs = CRS.from_proj4("+proj=lcc +lat_1=30 +lat_2=60 +lat_0=35 +lon_0=-100 +datum=WGS84")
counties = counties.to_crs(netcdf_crs)

# Plot wind speed with the correctly reprojected shapefile
plt.figure(figsize=(12, 8))
ax = plt.axes(projection=lambert_crs)

# Set extent using Lambert Conformal CRS
ax.set_extent([lons.min(), lons.max(), lats.min(), lats.max()], crs=ccrs.PlateCarree())

# Add geographic features
# ax.add_feature(cfeature.COASTLINE)
# ax.add_feature(cfeature.BORDERS, linestyle=':')
# ax.add_feature(cfeature.LAND, facecolor="lightgray")
# ax.add_feature(cfeature.OCEAN, facecolor="lightblue")

# Plot the wind speed
c = ax.pcolormesh(lons, lats, wind_speed_data, transform=ccrs.PlateCarree(), cmap="coolwarm")
cb = plt.colorbar(c, orientation="vertical", shrink=0.7, pad=0.05)
cb.set_label("Wind Speed (m/s)")

# Overlay the shapefile
for _, row in counties.iterrows():
    geometry = row['geometry']
    if geometry.geom_type == 'Polygon':
        ax.add_geometries([geometry], crs=lambert_crs, edgecolor='black', facecolor='none', linewidth=0.5)
    elif geometry.geom_type == 'MultiPolygon':
        for geom in geometry.geoms:
            ax.add_geometries([geom], crs=lambert_crs, edgecolor='black', facecolor='none', linewidth=0.5)

# Add title
plt.title("Total Wind Speed at 10m for October 1979", fontsize=14)
plt.show()


NameError: name 'wind_speed_data' is not defined

In [2]:
%%bash
ncdump -h /shared_space/ACV/extracted_raw/yearly_files/yearly2003.nc

netcdf yearly2003 {
dimensions:
	Time = UNLIMITED ; // (8760 currently)
	west_east = 236 ;
	south_north = 135 ;
variables:
	float Time(Time) ;
		Time:standard_name = "time" ;
		Time:units = "minutes since 1979-10-01 00:00:00" ;
		Time:calendar = "standard" ;
		Time:axis = "T" ;
	float XLONG(south_north, west_east) ;
		XLONG:standard_name = "longitude" ;
		XLONG:long_name = "longitude" ;
		XLONG:units = "degree_east" ;
		XLONG:_CoordinateAxisType = "Lon" ;
	float XLAT(south_north, west_east) ;
		XLAT:standard_name = "latitude" ;
		XLAT:long_name = "latitude" ;
		XLAT:units = "degree_north" ;
		XLAT:_CoordinateAxisType = "Lat" ;
	float U10(Time, south_north, west_east) ;
		U10:long_name = "U-component wind with respect to model grid at 10 meters" ;
		U10:units = "m s-1" ;
		U10:coordinates = "XLAT XLONG" ;
		U10:description = "U at 10 M" ;
	float V10(Time, south_north, west_east) ;
		V10:long_name = "V-component wind with respect to model grid at 10 meters" ;
		V10:units = "m s-1" ;
		V1